In [1]:
import time
import pandas as pd
from tqdm.notebook import tqdm
from selenium import webdriver
import numpy as np 
from pathlib import Path
from unidecode import unidecode
import requests, json

path = 'data'

root_folder = Path(path)
root_folder.mkdir(parents=True, exist_ok=True)

In [2]:
df_searchterms = pd.read_csv(root_folder /'medium_search_base.csv', sep = ';') 
list_dfs = []
for record in df_searchterms.to_dict(orient = 'records'):
    full_path_origin_folder = root_folder / 'raw_data'/ 'list_items' / record['folder']
    lst_jsons = list((full_path_origin_folder ).rglob("*.json")) 
    items = []
    for json_file_path in lst_jsons:
        if int(json_file_path.name.split('.json')[0]) > 940:
            continue
        with open(json_file_path, mode = 'rb') as file:
            r =file.read()
            json_file  = json.loads(r)
        items += json_file[0]['data']['search']['posts']['items']

    df = pd.DataFrame(items)
    for k,v in record.items():
        df[k] = v
    list_dfs.append(df)

df_searchterms = pd.concat(list_dfs, axis = 0, ignore_index=True)
df_searchterms.sort_values(by = ['folder','voterCount','mediumUrl'],\
                            ascending=[False,False,False], ignore_index=True, inplace=True)
df_searchterms.drop_duplicates(subset='algoliaObjectId',keep='first', inplace = True)
df_searchterms['path_folder'] = path + '\\articles\\'+ df_searchterms['folder']
df_searchterms['path_file'] = df_searchterms['path_folder']+ '\\'+df_searchterms['algoliaObjectId']+'.html'

In [3]:
df_searchterms.to_csv(root_folder /'medium_search_base_detail.csv', sep = ';')

In [4]:
driver = webdriver.Chrome()
url = df_searchterms['mediumUrl'].values[0]
driver.get(url)

In [10]:
records = df_searchterms.to_dict(orient = 'records')
records_filer = []

for record in records:

    full_path_folder = Path(record['path_folder'])
    full_path_folder.mkdir(parents=True, exist_ok=True)

    full_path = Path(record['path_file'])

    if full_path.exists() and full_path.is_file():  
        continue
    
    url = record['mediumUrl']
    if not isinstance(url, str):
        continue 

    records_filer.append(record)


for record in tqdm(records_filer):
    full_path_folder = Path(record['path_folder'])
    full_path_folder.mkdir(parents=True, exist_ok=True)

    full_path = Path(record['path_file'])
    url = record['mediumUrl']
    try:
        url = record['mediumUrl']
        driver.get(url)
        time.sleep(4)
        alpha, beta = 2,5
        val = np.random.beta(alpha, beta)
        wait_time = 1 + val * (3 - 1)
        time.sleep(wait_time) 

        if 'Confirme que você é humano' in driver.page_source:
            continue

        if 'Verificando se você é humano' in driver.page_source:
            continue

        if 'There was a problem providing the content you requested' in driver.page_source:
            continue

        with open(record['path_file'], "w", encoding="utf-8") as f:
            f.write(driver.page_source)
            
    except Exception as e:
            print('erro em ',url)
            print(e)
            error = True

    

  0%|          | 0/9 [00:00<?, ?it/s]

erro em  https://blog.ambrosus.com/partnerships-with-h-ve-honey-blendhub-and-flatev-514a49e7214
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=132.0.6834.160)
Stacktrace:
	GetHandleVerifier [0x00007FF6024102F5+28725]
	(No symbol) [0x00007FF602372AE0]
	(No symbol) [0x00007FF60220510A]
	(No symbol) [0x00007FF602202861]
	(No symbol) [0x00007FF6021F3559]
	(No symbol) [0x00007FF6021F52AF]
	(No symbol) [0x00007FF6021F381F]
	(No symbol) [0x00007FF6021F32ED]
	(No symbol) [0x00007FF6021F2FBA]
	(No symbol) [0x00007FF6021F0DD1]
	(No symbol) [0x00007FF6021F145C]
	(No symbol) [0x00007FF60220824A]
	(No symbol) [0x00007FF6022A0FAE]
	(No symbol) [0x00007FF60227FFAA]
	(No symbol) [0x00007FF6022A0181]
	(No symbol) [0x00007FF60227FD53]
	(No symbol) [0x00007FF60224A0E3]
	(No symbol) [0x00007FF60224B471]
	GetHandleVerifier [0x00007FF60273F30D+3366989]
	GetHandleVerifier [0x00007FF6027512F0+3440688]
	GetHandleVerifier [0x00007FF6027478FD+3401277]
	GetHandleVerifier [0x00007FF6024